In [1]:
import sys, os
sys.path.append('/usr/local/lib/python2.7/dist-packages/')
import pandas as pd
import dbconfig as cfg
from sqlalchemy import create_engine
import tensorflow as tf
import numpy as np
from collections import defaultdict
from random import shuffle

ModuleNotFoundError: No module named 'dbconfig'

In [2]:
engine = create_engine('mysql+pymysql://{}:{}@{}:3306/mimic'.format(cfg.mysql['user'], cfg.mysql['password'],
                                                                cfg.mysql['host']), echo=False)

In [3]:
PATIENTS = pd.read_sql_query(
    'SELECT SUBJECT_ID, GENDER, DOB, DOD, EXPIRE_FLAG, UNIQUE_ID FROM PATIENTS WHERE UNIQUE_ID IS NOT NULL', 
    engine, 
    index_col='SUBJECT_ID')
PATIENTS.head()

,GENDER,DOB,DOD,EXPIRE_FLAG,UNIQUE_ID
SUBJECT_ID,,,,,
3,M,2025-04-11 00:00:00,2102-06-14,1,2
4,F,2143-05-12 00:00:00,NaT,0,3
6,F,2109-06-21 00:00:00,NaT,0,4
9,M,2108-01-26 00:00:00,2149-11-14,1,5
11,F,2128-02-22 00:00:00,2178-11-14,1,6


In [4]:
ADMISSIONS = pd.read_sql_query(
    'SELECT \
        SUBJECT_ID, HADM_ID, ADMITTIME, DISCHTIME, DEATHTIME, \
        ADMISSION_TYPE, ADMISSION_LOCATION, HOSPITAL_EXPIRE_FLAG, UNIQUE_ID \
    FROM ADMISSIONS \
    WHERE UNIQUE_ID IS NOT NULL \
    ORDER BY SUBJECT_ID', 
    engine, index_col= 'SUBJECT_ID')
ADMISSIONS.head()

,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,HOSPITAL_EXPIRE_FLAG,UNIQUE_ID
SUBJECT_ID,,,,,,,,
3,145834,2101-10-20 19:08:00,2101-10-31 13:58:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,0,2
4,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,0,3
6,107064,2175-05-30 07:15:00,2175-06-15 16:00:00,NaT,ELECTIVE,PHYS REFERRAL/NORMAL DELI,0,4
9,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,2149-11-14 10:15:00,EMERGENCY,EMERGENCY ROOM ADMIT,1,5
11,194540,2178-04-16 06:18:00,2178-05-11 19:00:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,0,6


In [5]:
AGES = pd.read_sql_query(
    'SELECT T.SUBJECT_ID, T.AGE FROM mimic.temp2 T \
    JOIN PATIENTS P ON T.SUBJECT_ID = P.SUBJECT_ID;', 
    engine, index_col='SUBJECT_ID')
AGES.head()

,AGE
SUBJECT_ID,
3,76
4,47
6,65
9,41
11,50


In [ ]:
# MERGED_OUTPUTEVENTS = pd.read_sql_query('')

In [6]:
MERGED_LAB = pd.read_sql_table('MERGED_LAB', engine, index_col='SUBJECT_ID')
MERGED_LAB.head()

,COUNT,HADM_ID,ITEMID,CHARTTIME,VALUENUM,VALUEUOM,LABEL,CATEGORY,LOINC_CODE,ADMITTIME,TIMEDIFF
SUBJECT_ID,,,,,,,,,,,
3,172130,145834.0,51254,2101-10-21 03:00:00,5.0,%,Monocytes,Hematology,742-7,2101-10-20 19:08:00,7
3,172126,145834.0,51200,2101-10-21 03:00:00,0.0,%,Eosinophils,Hematology,711-2,2101-10-20 19:08:00,7
3,172124,145834.0,51146,2101-10-21 03:00:00,0.0,%,Basophils,Hematology,704-7,2101-10-20 19:08:00,7
3,146694,145834.0,50862,2101-10-20 19:59:00,1.8,g/dL,Albumin,Chemistry,1751-7,2101-10-20 19:08:00,0
3,132400,145834.0,50910,2101-10-20 19:59:00,82.0,IU/L,Creatine Kinase (CK),Chemistry,2157-6,2101-10-20 19:08:00,0


In [7]:
TOP_MERGED_CHARTEVENTS = pd.read_sql_table('TOP_MERGED_CHARTEVENTS', engine, index_col='SUBJECT_ID')
TOP_MERGED_CHARTEVENTS = TOP_MERGED_CHARTEVENTS.fillna(0)
TOP_MERGED_CHARTEVENTS.head()

,HADM_ID,ICUSTAY_ID,ITEMID,LABEL,CHARTTIME,VALUE,VALUENUM,VALUEUOM,TIMEDIFF
SUBJECT_ID,,,,,,,,,
3,145834,211552.0,113,CVP,2101-10-20 23:00:00,13,13.0,mmHg,3
3,145834,211552.0,113,CVP,2101-10-21 00:00:00,16,16.0,mmHg,4
3,145834,211552.0,113,CVP,2101-10-21 01:00:00,7,7.0,mmHg,5
3,145834,211552.0,113,CVP,2101-10-21 02:00:00,7,7.0,mmHg,6
3,145834,211552.0,113,CVP,2101-10-21 03:00:00,7,7.0,mmHg,7


In [8]:
# normalize to cohort
normal_dict = defaultdict(float)
labs_list = set(MERGED_LAB.LABEL.values)
chart_list = set(TOP_MERGED_CHARTEVENTS.LABEL.values)

for lab in labs_list:
    temp_df = MERGED_LAB[MERGED_LAB.LABEL == lab]
    normal_dict['lab_'+lab.lower()] = (min(temp_df.VALUENUM), max(temp_df.VALUENUM))

for lab in chart_list:
    temp_df = TOP_MERGED_CHARTEVENTS[TOP_MERGED_CHARTEVENTS.LABEL==lab]
    normal_dict[lab.lower()] = (min(temp_df.VALUENUM), max(temp_df.VALUENUM))

normal_dict

defaultdict(float,
            {'abp alarm [high]': (11.0, 300.0),
             'abp alarm [low]': (0.0, 215.0),
             'arterial blood pressure diastolic': (-16.0, 93156.0),
             'arterial blood pressure mean': (-135.0, 9381.0),
             'arterial blood pressure systolic': (-11.0, 411.0),
             'cvp': (0.0, 50.0),
             'eye opening': (0.0, 4.0),
             'gcs total': (0.0, 15.0),
             'lab_albumin': (1.0, 6.2999999999999998),
             'lab_amylase': (0.0, 22020.0),
             'lab_bacteria': (0.0, 0.0),
             'lab_bands': (0.0, 79.0),
             'lab_basophils': (0.0, 14.0),
             'lab_creatine kinase (ck)': (3.0, 266720.0),
             'lab_creatine kinase, mb isoenzyme': (0.90000000000000002,
              3320.0),
             'lab_eosinophils': (0.0, 55.600000000000001),
             'lab_epithelial cells': (0.0, 193.0),
             'lab_glucose': (70.0, 1000.0),
             'lab_hematocrit, calculated': (0.0, 3

In [30]:
d = {
   'chart_temp': 'temperature f',
    'bp_sys': 'non invasive blood pressure systolic',   
    'bp_dia': 'non invasive blood pressure diastolic',  
    'cvp': 'cvp',         
#     'resp': (16.0, 20.0),      #rate
    'art_sys': 'arterial blood pressure systolic', 
    'art_dia': 'arterial blood pressure diastolic',
     
    'lab_glucose': 'lab_glucose',  
    'lab_monocyte': 'lab_monocytes',   
    'lab_eosinophil': 'lab_eosinophils',  
    'lab_basophils': 'lab_basophils',
#     'lab_neutrohils': (40.0,80.0), #percent
#     'lab_lymphocytes': (20.0,40.0),#percent
    'lab_hemoglobin': 'lab_hemoglobin',  
    'lactate_dh': 'lab_lactate dehydrogenase (ld)',
    'sodium_whole_blood': 'lab_sodium, whole blood',
    'troponin_t': 'lab_troponin t',
    'lab_ck': 'lab_creatine kinase (ck)'
}
def normalize(value, event, patient):
    errors = set()
    try:
        event = d[event]
    except:
        errors.add(event)
        return value
    low = normal_dict[event][0]
    high = normal_dict[event][1]
    return (value-low)/(high-low)
    
def normalize_timeseries(val_list, event, patient):
    n_list = []
    for t in val_list:
        if t==0:
            n_list.append(0)
            continue
        n_list.append(normalize(t, event, patient))
    return n_list


In [42]:
def standardize(value, event, patient):  
    if 'ck' in event:
        if AGES.at[patient, 'AGE'] == 1:
            event = 'lab_ck_female'
        else:
            event = 'lab_ck_male'
    if 'temp' in event: # adjust for different temp sites
        event = 'temp'  # use general for now
    if 'hemoglobin' in event:
        if AGES.at[patient, 'AGE'] == 1:
            event = 'lab_hemoglobin_female'
        else:
            event = 'lab_hemoglobin_male'
    if event not in range_dict:
        return value
    low = range_dict[event][0]
    high = range_dict[event][1]
    return (value-low)/(high-low)

In [43]:
# expert defined ranges (low, high) for standardization
range_dict = {
    'temp': (97.8, 99.0),
    'temp_aux': (97.4, 98.7), #auxillary/armpit, (F)
    'temp_rectal': (98.3, 100.0), 
    'bp_sys': (120.0,180.0),   #mmHg
    'bp_dia': (80.0, 120.0),   #mmHg
    'cvp': (3.0, 8.0),         #
    'resp': (16.0, 20.0),      #rate
    'art_sys': (20.0, 119.0),  #mmHg
    'art_dia': (60.0, 79.0),   #mmHg
     
    'lab_ck_male': (52.0, 336.0),  #U/L
    'lab_ck_female': (38.0,176.0), #U/L
    'lab_glucose': (70.0, 100.0),  #mg/dL
    'lab_monocyte': (2.0, 10.0),   #percent
    'lab_eosinophil': (1.0, 6.0),  #percent
    'lab_basophils': (0.01, 2.0),#percent
#     'lab_neutrohils': (40.0,80.0), #percent
    'lab_lymphocytes': (20.0,40.0),#percent
    'lab_hemoglobin_female': (12.0, 17.5),  #g/dL
    'lab_hemoglobin_male': (13.5, 17.5),   #g/dL
    'lactate_dh': (140.0, 280.0), #U/L
    'sodium_whole_blood': (135.0, 145.0), #mEq/L
    'troponin_t': (0, 0.01) #ng/mL
}

In [44]:
def standardize_timeseries(val_list, event, patient):
    n_list = []
    for t in val_list:
        if t == 0:  #missing
            n_list.append(0)
            continue
        n_list.append(standardize(t, event, patient))
    return n_list
        

In [10]:
def get_expire_flag(subject_id):
    return ADMISSIONS.get_value(subject_id, 'HOSPITAL_EXPIRE_FLAG')

In [11]:
def get_age(subject_id):
    return AGES.get_value(subject_id, 'AGE')

In [12]:
def get_gender(subject_id):
    if PATIENTS.get_value(subject_id, 'GENDER') == 'F':
        return 1
    return 0

In [13]:
def get_lab_events(subject_id):
    d = defaultdict(lambda: defaultdict(list))
    temp = MERGED_LAB[MERGED_LAB.index == subject_id]
    for index, row in temp.iterrows():
#         print(row.ITEMID, row.VALUENUM, row.LABEL, row.TIMEDIFF)
        d[row.ITEMID][row.TIMEDIFF].append(row.VALUENUM)
    return d

In [14]:
def get_lab_timeseries(subject_id, itemid, lab_dict):
    l = [0]*24
    for i, hour in enumerate(sorted(lab_dict[itemid])):
        l[hour] = int(np.mean(lab_dict[itemid][hour]))
    return l

In [15]:
def get_lab_timeseries_mean(subject_id, itemid, lab_dict):
    l = []
    for hour in lab_dict[itemid]:
        l.extend(lab_dict[itemid][hour])
    if not l:
        return 0
    return int(np.mean(l))

In [16]:
def get_output_timeseries(subject_id, itemid, out_dict):
    l = [0]*24
    for i, hour in enumerate(sorted(out_dict[itemid])):
        l[hour] = int(np.mean(lab_dict[itemid][hour]))
    return l

In [17]:
def get_output_timeseries_mean(subject_id, itemid, out_dict):
    l = []
    for hour in out_dict[itemid]:
        l.extend(out_dict[itemid][hour])
    if not l:
        return 0
    return int(np.mean(l))

In [18]:
def get_chartevents(subject_id):
    d = defaultdict(lambda: defaultdict(list))    
    temp = TOP_MERGED_CHARTEVENTS[TOP_MERGED_CHARTEVENTS.index == subject_id]
    for index, row in temp.iterrows():
        if row.VALUENUM:
            d[row.ITEMID][row.TIMEDIFF].append(row.VALUENUM)
    return d

In [19]:
def get_chart_timeseries(subject_id, itemid, chart_dict):
    l = [0]*24
    for hour in chart_dict[itemid]:
        l[hour] = int(np.mean(chart_dict[itemid][hour]))
    return l

In [20]:
def get_chart_timeseries_mean(subject_id, itemid, chart_dict):
    l = []
    for hour in chart_dict[itemid]:
        l.extend(chart_dict[itemid][hour])
    if not l:
        return 0
    return int(np.mean(l))

### Make and save dataframe for timeseries and non-timeseries model

In [21]:
def make_dataframe(patient_list, labdict, chartdict, timeseries, normalized=False):
    labdf = pd.DataFrame(index=patient_list, columns=labdict.keys())
    for patient in patient_list:
        temp_labdict = get_lab_events(patient)
        for l in labdict.keys(): 
            if timeseries:
                if not normalized:
                    labdf.at[patient, l] = standardize_timeseries(get_lab_timeseries(patient, labdict[l], temp_labdict), l, patient)
                else:
                    labdf.at[patient, l] = normalize_timeseries(get_lab_timeseries(patient, labdict[l], temp_labdict), l, patient)
            else:
                if not normalized:
                    labdf.at[patient, l] = standardize(get_lab_timeseries_mean(patient, labdict[l], temp_labdict), l, patient)
                else:
                    labdf.at[patient, l] = normalize(get_lab_timeseries_mean(patient, labdict[l], temp_labdict), l, patient)
    chartdf = pd.DataFrame(index=patient_list, columns=chartdict.keys())
    for patient in patient_list:
        temp_chartdict=get_chartevents(patient)
        for c in chartdict.keys():
            if timeseries:
                if not normalized:
                    chartdf.at[patient, c] = standardize_timeseries(get_chart_timeseries(patient, chartdict[c], temp_chartdict), c, patient)
                else:
                    chartdf.at[patient, c] = normalize_timeseries(get_chart_timeseries(patient, chartdict[c], temp_chartdict), c, patient)
            else:
                if not normalized:
                    chartdf.at[patient, c] = standardize(get_chart_timeseries_mean(patient, chartdict[c], temp_chartdict), c, patient)
                else:
                    chartdf.at[patient, c] = normalize(get_chart_timeseries_mean(patient, chartdict[c], temp_chartdict), c, patient)
    outputdf = PATIENTS['GENDER'].copy()
    # TEST = pd.concat([TEST, AGES], axis=1)
    outputdf = pd.concat([outputdf, AGES, ADMISSIONS.ADMISSION_LOCATION, ADMISSIONS.ADMISSION_TYPE, ADMISSIONS.HOSPITAL_EXPIRE_FLAG], axis=1)
    outputdf = pd.concat([outputdf, labdf], axis=1)
    outputdf = pd.concat([outputdf, chartdf], axis=1)
    return outputdf

In [22]:
patient_list = PATIENTS.index.values
labdict = {'lab_ck': 50910, 'lab_monocyte': 51254, 'lab_glucose':51478 , 'lab_eosinophil': 51200, 
           'lab_basophils': 51146, 'lab_hemoglobin': 50811, 'lactate_dh': 50954, 'sodium_whole_blood': 50824, 
           'troponin_t': 51003} # 'lab_temp': 50825,
chartdict = {'bp_sys': 220179, 'bp_dia': 220180, 'cvp': 113, 'art_mean': 220052, 'art_sys': 220050, 
             'art_dia': 220051, 'riker_sas': 1337, 'resp_pattern': 617, 'eye_open': 184, 'glasgow_score': 198, 
             'chart_temp': 678} #  'abp_low': 5813, 'abp_high': 8547,

In [31]:
# timeseries_df = make_dataframe(patient_list, labdict, chartdict, timeseries=True, normalized=True)
timeseries_df.head()

,GENDER,AGE,ADMISSION_LOCATION,ADMISSION_TYPE,HOSPITAL_EXPIRE_FLAG,lab_hemoglobin,lab_monocyte,lactate_dh,lab_eosinophil,lab_glucose,...,resp_pattern,bp_dia,chart_temp,glasgow_score,art_mean,bp_sys,art_sys,riker_sas,cvp,eye_open
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,M,76,EMERGENCY ROOM ADMIT,EMERGENCY,0,"[0, 0, 0.0769230769231, 0, 0, 0, 0.07692307692...","[0, 0, 0, 0, 0, 0, 0, 0.0641025641026, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0.0322580645161, 0, 0...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.880733944954, 0, 0, 0, 0, 0.899082568807, 0...","[0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0.26, 0.32, 0.14, 0.14, 0.14, 0.22, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ..."
4,F,47,EMERGENCY ROOM ADMIT,EMERGENCY,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0.025641025641, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0.00406604917852, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0.908256880734, 0, 0, 0, 0, 0, 0.899082568...","[0, 15, 0, 0, 0, 0, 0, 15, 0, 0, 0, 15, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 0, 4, 0, 0, 0, ..."
6,F,65,PHYS REFERRAL/NORMAL DELI,ELECTIVE,0,"[0.0615384615385, 0, 0, 0.0615384615385, 0.069...","[0, 0, 0, 0, 0, 0, 0, 0, 0.0128205128205, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.8807...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 15...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, ..."
9,M,41,EMERGENCY ROOM ADMIT,EMERGENCY,1,"[0, 0, 0, 0, 0.107692307692, 0, 0, 0, 0, 0, 0,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.871559633028, 0, 0.880733944954, 0, 0, 0, 0...","[7, 7, 7, 7, 7, 0, 3, 0, 3, 7, 7, 7, 3, 7, 7, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
11,F,50,EMERGENCY ROOM ADMIT,EMERGENCY,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.899082568807, 0, 0, 0, 0.8

In [32]:
non_timeseries_df = make_dataframe(patient_list, labdict, chartdict, timeseries=False, normalized=True)
non_timeseries_df.head()

,GENDER,AGE,ADMISSION_LOCATION,ADMISSION_TYPE,HOSPITAL_EXPIRE_FLAG,lab_hemoglobin,lab_monocyte,lactate_dh,lab_eosinophil,lab_glucose,...,resp_pattern,bp_dia,chart_temp,glasgow_score,art_mean,bp_sys,art_sys,riker_sas,cvp,eye_open
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,M,76,EMERGENCY ROOM ADMIT,EMERGENCY,0,0.0769231,0.0641026,-0.000482413,0,0.0322581,...,0,0,0.889908,4,0,0,0.0260664,0,0.22,1
4,F,47,EMERGENCY ROOM ADMIT,EMERGENCY,0,0,0.025641,0.00406605,0,-0.0752688,...,0,0,0.899083,15,0,0,0.0260664,0,0,4
6,F,65,PHYS REFERRAL/NORMAL DELI,ELECTIVE,0,0.0692308,0.0128205,-0.000482413,0,-0.0752688,...,0,0,0.889908,15,0,0,0.0260664,0,0.24,4
9,M,41,EMERGENCY ROOM ADMIT,EMERGENCY,1,0.107692,0,-0.000482413,0,-0.0752688,...,0,0,0.899083,5,0,0,0.0260664,0,0,1
11,F,50,EMERGENCY ROOM ADMIT,EMERGENCY,0,0,0,-0.000482413,0,-0.0752688,...,0,0,0.899083,14,0,0,0.0260664,0,0,4


In [33]:
non_timeseries_df.to_pickle('/home/andrea/data/mimic_nontimeseries_normalized')
timeseries_df.to_pickle('/home/andrea/data/mimic_timeseries_normalized')